# Hyperparameter Tuning 과제

- Manual Search
- Random Search (Grid Search 참고)
- Bayesian Optimization using HyperOpt
<br>연습해보기!

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.datasets import load_iris
import warnings
warnings.filterwarnings('ignore')
import time
from functools import partial

In [2]:
iris = load_iris()

X = iris.data
y = iris.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

## 1. Manual Search
사이킷런 Logistic Regresstion https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression

In [4]:
manual = LogisticRegression(
   penalty='l2', dual=False, tol=0.0001 , class_weight=None, 
    C=2, fit_intercept=True, intercept_scaling=1, 
    random_state=0, max_iter=100)
manual.fit(X_train,y_train)
# 정확도까지 계산해주세요
y_pred = manual.predict(X_train)

print(accuracy_score(y_train, y_pred))
print(classification_report(y_train, y_pred))

0.9821428571428571
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        34
           1       1.00      0.95      0.97        39
           2       0.95      1.00      0.97        39

    accuracy                           0.98       112
   macro avg       0.98      0.98      0.98       112
weighted avg       0.98      0.98      0.98       112



## 2. Grid Search

In [5]:
from sklearn.model_selection import GridSearchCV

# 파라미터 후보 설정
## C: 표준화 정도
## max_iter: 최대 반복 횟수
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'max_iter' : [100, 500, 1000, 1500, 2000]}

# 그리드 서치 호출
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, return_train_score=True)   

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

start = time.time()
# 모델 적합
grid_search.fit(X_train, y_train)

# 모델 평가
print(f"test set score : {grid_search.score(X_test, y_test)}")

print(f"best parameters: {grid_search.best_params_}")
print(f"best CV score: {grid_search.best_score_}")

# 소요 시간 측정
print("time :", time.time() - start)

test set score : 1.0
best parameters: {'C': 1, 'max_iter': 100}
best CV score: 0.9553359683794467
time : 3.3071553707122803


## 3. Random Search
grid search 코드와 매우매우 유사합니다

In [6]:
from sklearn.model_selection import RandomizedSearchCV

# 파라미터 후보 설정
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'max_iter' : [100, 500, 1000, 1500, 2000]}

# 랜덤 서치 호출
random_search = RandomizedSearchCV(LogisticRegression(), param_grid, cv=5, return_train_score=True)   

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

start = time.time()
# 모델 적합
random_search.fit(X_train, y_train)

# 모델 평가
print(f"test set score : {grid_search.score(X_test, y_test)}")

print(f"best parameters: {grid_search.best_params_}")
print(f"best CV score: {grid_search.best_score_}")

# 소요 시간 측정
print("time :", time.time() - start)

test set score : 1.0
best parameters: {'C': 1, 'max_iter': 100}
best CV score: 0.9553359683794467
time : 1.3643980026245117


## 4. HyperOpt

In [7]:
!pip install hyperopt

You should consider upgrading via the 'c:\users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [16]:

space = {
    'penalty' : hp.choice('penalty', ['l1', 'l2', 'elasticnet', 'none']), 
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'max_iter' : hp.uniform('max_iter', 100, 1000),
}

def objective(space):
    model = LogisticRegression(penalty = space['penalty'],
                               fit_intercept = space['fit_intercept'],
                               max_iter = space['max_iter'],
                              )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 4).mean() 

    return {'loss': -accuracy, 'status': STATUS_OK }
    
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 20,
            trials = trials)
best

100%|███████████████████████████████████████████████| 20/20 [00:00<00:00, 26.66trial/s, best loss: -0.9553571428571429]


{'fit_intercept': 0, 'max_iter': 328.67439188549105, 'penalty': 3}

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

pen = {0: 'l1', 1: 'l2', 2: 'elasticnet', 3: None}
incpt = {0: True, 1: False}

trained = LogisticRegression(penalty = pen[best['penalty']],
                             fit_intercept = incpt[best['fit_intercept']],
                             max_iter = best['max_iter'],
                            ).fit(X_train,y_train)

# 정확도 도출 부분까지 코드 짜시면 됩니다!
hyopt = trained.predict(X_test)
print(confusion_matrix(y_test, hyopt))
print(classification_report(y_test,hyopt))
hyopt_acc = accuracy_score(y_test,htopt)

## 선택) Bayesian Optimization 사용


In [ ]:
!pip install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization